In [1]:
!pip install --quiet xlrd
print('pip install xlrd complete')

pip install xlrd complete


In [2]:
import pandas as pd
import numpy as np


DATA = '/kaggle/input/university-students-marks-sheet/Univeristy_Results.xls'

sheet_names = ['Final', 'Sessional', 'Table']

final_df = pd.read_excel(io=DATA, sheet_name=sheet_names[0], engine='xlrd', )
session_df = pd.read_excel(io=DATA, sheet_name=sheet_names[1], engine='xlrd',)
table_df = pd.read_excel(io=DATA, sheet_name=sheet_names[2], engine='xlrd',)
print(session_df.columns)
session_df.columns = ['roll', 'name', 'Q1', 'Q2', 'AV1', 'A1', 'A2', 'AV2', 'M', 'Total']
session_df = session_df[session_df.index != 0].head(216)
numeric_columns = [item for item in session_df.columns if item != 'name']
session_df['roll'] = session_df['roll'].apply(func=lambda x: x if isinstance(x, float) or isinstance(x, int) or x.strip() == '' else x)
for column in numeric_columns:
    session_df[column] = session_df[column].apply(func=lambda x: x.strip() if isinstance(x, str) else x)
session_df[numeric_columns] = session_df[numeric_columns].replace('', '0').astype(float).fillna(value=0)
session_df.head()

Index(['Roll No', 'Name', 'Quiz 15 ', 'Unnamed: 3', 'Unnamed: 4',
       'Assignments 15 ', 'Unnamed: 6', 'Unnamed: 7', 'Mid Term', 'Q+A+Mid'],
      dtype='object')


,roll,name,Q1,Q2,AV1,A1,A2,AV2,M,Total
1,1.0,Ashir Mehfooz,14.0,14.0,14.0,13.0,13.0,13.0,41.0,68.0
2,2.0,Atif Raftad,4.0,10.0,7.0,4.0,5.0,4.5,30.0,41.5
3,3.0,Saiqa Aziz,15.0,11.0,13.0,14.0,13.0,13.5,34.0,60.5
4,8.0,Ozair Minhas,6.0,5.0,5.5,4.0,6.0,5.0,12.0,22.5
5,9.0,Naveera Subhani,5.0,11.0,8.0,4.0,5.0,4.5,34.0,46.5


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

scores = ['Q1', 'Q2', 'AV1', 'A1', 'A2', 'AV2', 'M',] 
X_train, X_test, y_train, y_test = train_test_split(session_df[scores], session_df['Total'], test_size=0.25, random_state=2024)

linear = LinearRegression()
linear.fit(X_train, y_train)
print('r2: {:5.4f}'.format(r2_score(y_true=y_test, y_pred=linear.predict(X_test))))


r2: 0.9458


In [4]:
from plotly.express import histogram
histogram(x=scores, y=linear.coef_)